Testing for Cigen v2- esta el la copia, solo usar para h2o en ccpvdz
====

In [ ]:
import main_fci_lstm
import mlflow
import itertools

python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
torch version: 2.4.1+cu121


In [ ]:

#----------------hyperparameter grid search----------------
max_iterations=4 #note, scf y cisd cuentan como una iteracion
qpsh_path='/home/ivan/Descargas/qp2/bin/qpsh'

num_epochs=[0,0,0,1,1,1,2,2,2,4,4,4,8,8,8,16,16,16] #number of epochs for each iteration
learning_rate=[0.0005]
times_num_dets_gen=[15]  #how many times the number of determinants generated
batch_size=[64]
embedding_dim=[8]
n_qubits=[4]
prune=[1e-12]
tol=[1e-8]    #tolerance for the energy convergence in hartrees. for the moment, it is fixed to 1e-8 to avoid that the program converges before the max_iterations
#FCI_energy=-76.12237    #h2o 6-31g

In [7]:
#generate all the combinations of hyperparameters
hyperparameters_combination=list(itertools.product(num_epochs,learning_rate,times_num_dets_gen,prune,tol,batch_size,embedding_dim,n_qubits))

In [8]:
len(hyperparameters_combination)

18

In [ ]:
#create a experiment in mlflow called experiment_lstm
#mlflow.set_experiment('experiment_qlstm_epochs')

2025/05/13 21:27:20 INFO mlflow.tracking.fluent: Experiment with name 'experiment_qlstm_epochs' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/ivan/Descargas/Python_Codes_DFT/paper_code_implementation/lstm_fci/mlruns/148878683126056441', creation_time=1747193240983, experiment_id='148878683126056441', last_update_time=1747193240983, lifecycle_stage='active', name='experiment_qlstm_epochs', tags={}>

In [ ]:
#experiment with 0 epochs to use as a baseline


Hyperparameter search for H2O in 631g Basis
===

In [9]:
run_name = "h2o_631g"
ezfio_path = '/home/ivan/Descargas/QP_examples/h2o/h2o_631g.ezfio'
working_directory='/home/ivan/Descargas/Python_Codes_DFT/paper_code_implementation/lstm_fci'
FCI_energy = -76.12237 
current_run = 0
times_max_diag_time = 100
max_iterations = 5
# hyperparameters_combination starting from 22
for params in hyperparameters_combination:
    print('*****************************************current run:', current_run, 'out of', len(hyperparameters_combination), '********************************************')

    experiment_name = "experiment_qlstm"
    experiment = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id
    ezfio_name=ezfio_path.split('/')[-1]

    with mlflow.start_run(experiment_id= experiment_id, run_name=run_name) as run:

        # this two hyperparameters are fixed
         
        qpsh_path = '/home/ivan/Descargas/qp2/bin/qpsh'

        # Set the hyperparameters for the run
        num_epochs,learning_rate,times_num_dets_gen,prune,tol,batch_size,embedding_dim,n_qubits = params
        #num_epochs = 1  #0 epochs for the baseline
        #prune = 1e-12 #prune is fixed to 1e-8 for the baseline
        #batch_size = 64 #batch size is fixed to 128 for the baseline
        #embedding_dim = 8

        # run the model
        ground_energy_list,number_of_det_list,times_per_iteration_list=main_fci_lstm.main(working_directory,ezfio_path,qpsh_path,max_iterations,num_epochs, learning_rate, 
                                                                        times_num_dets_gen,prune,tol,FCI_energy,times_max_diag_time, batch_size, embedding_dim,n_qubits)
        
        final_energy_approx = ground_energy_list[-1]

        # save the parameters of the model in MLflow
        mlflow.log_param("max_iterations", max_iterations)
        mlflow.log_param("ezfio_path", ezfio_path)
        mlflow.log_param("num_epochs", num_epochs)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("times_num_dets_gen", times_num_dets_gen)
        mlflow.log_param("prune", prune)
        mlflow.log_param("tol", tol)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("embedding_dim", embedding_dim)
        mlflow.log_param("n_qubits", n_qubits)
    

        

        #save image of the results
        for iteration in range(1, len(ground_energy_list)-1):
            mlflow.log_artifact("graphs/det_distribution/det_distribution_"+ezfio_name+"_iteration_"+str(iteration)+".png")

        mlflow.log_artifact('graphs/energy_plot/'+ezfio_name+'_ground_energy.png')
        mlflow.log_artifact('graphs/time_per_dets/'+ezfio_name+'_time_per_dets.png')

        #save the results of the model in MLflow
        #mlflow.log_metric("ground_energy_list", ground_energy_list)
        for i, ground_energy in enumerate(ground_energy_list):
            mlflow.log_metric("ground_energy_"+str(i), ground_energy)

        #mlflow.log_metric("number_of_det_list", number_of_det_list)
        for i, number_of_det in enumerate(number_of_det_list):
            mlflow.log_metric("number_of_det_"+str(i), number_of_det)
        
        #mlflow.log_metric("times_per_iteration_list", times_per_iteration_list)
        for i, time_per_iteration in enumerate(times_per_iteration_list):
            mlflow.log_metric("times_per_iteration_"+str(i), time_per_iteration)

        mlflow.log_metric("final_energy_approx", final_energy_approx)
    
        current_run += 1
        

*****************************************current run: 0 out of 18 ********************************************
Number of electrons: 10
Number of molecular orbitals(alphas + betas): 26
Current working directory: /home/ivan/Descargas/Python_Codes_DFT/paper_code_implementation/lstm_fci
The selected ezfio_path is : /home/ivan/Descargas/QP_examples/h2o/h2o_631g.ezfio
current qpsh path: /home/ivan/Descargas/qp2/bin/qpsh
**********************************************************
************ iteration: 0 ***************************
**********************************************************
Resetting ezfio file finished
SCF started....
EL EZFIO /home/ivan/Descargas/QP_examples/h2o/h2o_631g.ezfio
EL QPSH /home/ivan/Descargas/qp2/bin/qpsh
Commands to execute in the interactive shell: qp set_file /home/ivan/Descargas/QP_examples/h2o/h2o_631g.ezfio
qp run scf |tee /home/ivan/Descargas/QP_examples/h2o/h2o_631g.ezfio/determinants/qp.out
qp unset_file h2o_631g.ezfio
exit

SCF finished
CISD started...

gzip: /home/ivan/Descargas/QP_examples/h2o/h2o_631g.ezfio/determinants/psi_coef.gz: No such file or directory
gzip: /home/ivan/Descargas/QP_examples/h2o/h2o_631g.ezfio/determinants/psi_det.gz: No such file or directory


x_train shape: 26
Shape after expand_dims: (679, 26)
Training the QLSTM Autoencoder...
Model initialized
Optimizer initialized
Model parameters saved to model_params.pkl
proceso terminado
predicting with qlstm


/home/ivan/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/backend/fork_exec.py:37: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
torch version: 2.4.1+cu121
python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
torch version: 2.4.1+cu121
python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
torch version: 2.4.1+cu121
python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
torch version: 2.4.1+cu121
python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
torch version: 2.4.1+cu121
python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
torch version: 2.4.1+cu121
python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
torch version: 2.4.1+cu121
python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
torch version: 2.4.1+cu121
python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
torch version: 2.4.1+cu121
python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
torch version: 2.4.1+cu121
python version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 1

...

<Figure size 1920x1440 with 0 Axes>

<Figure size 1920x1440 with 0 Axes>

<Figure size 1920x1440 with 0 Axes>

<Figure size 1920x1440 with 0 Axes>

<Figure size 1920x1440 with 0 Axes>

Hyperparameter search for H2O in cc-pvdz Basis
===

In [12]:
run_name = "h2o_ccpvdz"
ezfio_path = '/home/ivan/Descargas/QP_examples/h2o/h2o_ccpvdz.ezfio'
FCI_energy = -76.24195
current_run = 51
for params in hyperparameters_combination[251:]:
    print('*****************************************current run:', current_run, 'out of', len(hyperparameters_combination), '********************************************')

    experiment_name = "experiment_cigen_v3"
    experiment = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id
    ezfio_name=ezfio_path.split('/')[-1]

    with mlflow.start_run(experiment_id= experiment_id, run_name=run_name) as run:

        # this two hyperparameters are fixed
        max_iterations = 10  
        qpsh_path = '/home/ivan/Descargas/qp2/bin/qpsh'

        # Set the hyperparameters for the run
        num_epochs, batch_size, learning_rate, k, times_num_dets_gen, prune, tol, temp = params

        # run the model
        ground_energy_list, number_of_det_list, times_per_iteration_list = cigen.main(ezfio_path, qpsh_path, max_iterations, 
            num_epochs, batch_size, learning_rate, k, times_num_dets_gen, prune, tol, temp, FCI_energy)
        
        final_energy_approx = ground_energy_list[-1]

        # save the parameters of the model in MLflow
        mlflow.log_param("max_iterations", max_iterations)
        mlflow.log_param("ezfio_path", ezfio_path)
        mlflow.log_param("num_epochs", num_epochs)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("k", k)
        mlflow.log_param("times_num_dets_gen", times_num_dets_gen)
        mlflow.log_param("prune", prune)
        mlflow.log_param("tol", tol)
        mlflow.log_param("temp", temp)

        

        #save image of the results
        for iteration in range(1, len(ground_energy_list)-1):
            mlflow.log_artifact("graphs/det_distribution/det_distribution_"+ezfio_name+"_iteration_"+str(iteration)+".png")

        mlflow.log_artifact('graphs/energy_plot/'+ezfio_name+'_ground_energy.png')
        mlflow.log_artifact('graphs/time_per_dets/'+ezfio_name+'_time_per_dets.png')

        #save the results of the model in MLflow
        #mlflow.log_metric("ground_energy_list", ground_energy_list)
        for i, ground_energy in enumerate(ground_energy_list):
            mlflow.log_metric("ground_energy_"+str(i), ground_energy)

        #mlflow.log_metric("number_of_det_list", number_of_det_list)
        for i, number_of_det in enumerate(number_of_det_list):
            mlflow.log_metric("number_of_det_"+str(i), number_of_det)
        
        #mlflow.log_metric("times_per_iteration_list", times_per_iteration_list)
        for i, time_per_iteration in enumerate(times_per_iteration_list):
            mlflow.log_metric("times_per_iteration_"+str(i), time_per_iteration)

        mlflow.log_metric("final_energy_approx", final_energy_approx)
    
        current_run += 1

*****************************************current run: 51 out of 288 ********************************************
ezfio_path: /home/ivan/Descargas/QP_examples/h2o/h2o_ccpvdz.ezfio
**********************************************************
************ iteration: 0 ***************************
**********************************************************
Resetting ezfio file....
ezfio_name h2o_ccpvdz.ezfio
qp reset_ezfio executed
Resetting ezfio file finished
SCF started....
qp scf executed
SCF finished
Ground Energy: -76.02676814094778
CISD started....
qp cisd executed
CISD finished
Ground Energy: -76.23201494252775
**********************************************************
************ iteration: 1 ***************************
**********************************************************
Unzipping files ....
number of coefficients before pruning 3416
number of coefficients after pruning 3239
Number of orbitals: 48
numero de electrones 10
number of determinants generated 9717
Number of determin

Hyperparameter search for C2 in 631g Basis
===

In [ ]:
run_name = "c2_631g"
ezfio_path = '/home/ivan/Descargas/QP_examples/c2/c2_631g.ezfio'
FCI_energy = -75.64418
current_run = 0
for params in hyperparameters_combination:
    print('*****************************************current run:', current_run, 'out of', len(hyperparameters_combination), '********************************************')

    experiment_name = "experiment_cigen_v2"
    experiment = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id
    ezfio_name=ezfio_path.split('/')[-1]

    with mlflow.start_run(experiment_id= experiment_id, run_name=run_name) as run:

        # this two hyperparameters are fixed
        max_iterations = 10  
        qpsh_path = '/home/ivan/Descargas/qp2/bin/qpsh'

        # Set the hyperparameters for the run
        num_epochs, batch_size, learning_rate, k, times_num_dets_gen, prune, tol, temp = params

        # run the model
        ground_energy_list, number_of_det_list, times_per_iteration_list = cigen.main(ezfio_path, qpsh_path, max_iterations, 
            num_epochs, batch_size, learning_rate, k, times_num_dets_gen, prune, tol, temp, FCI_energy)
        
        final_energy_approx = ground_energy_list[-1]

        # save the parameters of the model in MLflow
        mlflow.log_param("max_iterations", max_iterations)
        mlflow.log_param("ezfio_path", ezfio_path)
        mlflow.log_param("num_epochs", num_epochs)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("k", k)
        mlflow.log_param("times_num_dets_gen", times_num_dets_gen)
        mlflow.log_param("prune", prune)
        mlflow.log_param("tol", tol)
        mlflow.log_param("temp", temp)

        

        #save image of the results
        for iteration in range(1, len(ground_energy_list)-1):
            mlflow.log_artifact("graphs/det_distribution/det_distribution_"+ezfio_name+"_iteration_"+str(iteration)+".png")

        mlflow.log_artifact('graphs/energy_plot/'+ezfio_name+'_ground_energy.png')
        mlflow.log_artifact('graphs/time_per_dets/'+ezfio_name+'_time_per_dets.png')

        #save the results of the model in MLflow
        #mlflow.log_metric("ground_energy_list", ground_energy_list)
        for i, ground_energy in enumerate(ground_energy_list):
            mlflow.log_metric("ground_energy_"+str(i), ground_energy)

        #mlflow.log_metric("number_of_det_list", number_of_det_list)
        for i, number_of_det in enumerate(number_of_det_list):
            mlflow.log_metric("number_of_det_"+str(i), number_of_det)
        
        #mlflow.log_metric("times_per_iteration_list", times_per_iteration_list)
        for i, time_per_iteration in enumerate(times_per_iteration_list):
            mlflow.log_metric("times_per_iteration_"+str(i), time_per_iteration)

        mlflow.log_metric("final_energy_approx", final_energy_approx)
    
        current_run += 1

Hyperparameter search for C2 in 631g Basis
===

In [ ]:
run_name = "c2_ccpvdz"
ezfio_path = '/home/ivan/Descargas/QP_examples/c2/c2_ccpvdz.ezfio'
FCI_energy = -75.72984
current_run = 0
for params in hyperparameters_combination:
    print('*****************************************current run:', current_run, 'out of', len(hyperparameters_combination), '********************************************')

    experiment_name = "experiment_cigen_v2"
    experiment = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id
    ezfio_name=ezfio_path.split('/')[-1]

    with mlflow.start_run(experiment_id= experiment_id, run_name=run_name) as run:

        # this two hyperparameters are fixed
        max_iterations = 10  
        qpsh_path = '/home/ivan/Descargas/qp2/bin/qpsh'

        # Set the hyperparameters for the run
        num_epochs, batch_size, learning_rate, k, times_num_dets_gen, prune, tol, temp = params

        # run the model
        ground_energy_list, number_of_det_list, times_per_iteration_list = cigen.main(ezfio_path, qpsh_path, max_iterations, 
            num_epochs, batch_size, learning_rate, k, times_num_dets_gen, prune, tol, temp, FCI_energy)
        
        final_energy_approx = ground_energy_list[-1]

        # save the parameters of the model in MLflow
        mlflow.log_param("max_iterations", max_iterations)
        mlflow.log_param("ezfio_path", ezfio_path)
        mlflow.log_param("num_epochs", num_epochs)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("k", k)
        mlflow.log_param("times_num_dets_gen", times_num_dets_gen)
        mlflow.log_param("prune", prune)
        mlflow.log_param("tol", tol)
        mlflow.log_param("temp", temp)

        

        #save image of the results
        for iteration in range(1, len(ground_energy_list)-1):
            mlflow.log_artifact("graphs/det_distribution/det_distribution_"+ezfio_name+"_iteration_"+str(iteration)+".png")

        mlflow.log_artifact('graphs/energy_plot/'+ezfio_name+'_ground_energy.png')
        mlflow.log_artifact('graphs/time_per_dets/'+ezfio_name+'_time_per_dets.png')

        #save the results of the model in MLflow
        #mlflow.log_metric("ground_energy_list", ground_energy_list)
        for i, ground_energy in enumerate(ground_energy_list):
            mlflow.log_metric("ground_energy_"+str(i), ground_energy)

        #mlflow.log_metric("number_of_det_list", number_of_det_list)
        for i, number_of_det in enumerate(number_of_det_list):
            mlflow.log_metric("number_of_det_"+str(i), number_of_det)
        
        #mlflow.log_metric("times_per_iteration_list", times_per_iteration_list)
        for i, time_per_iteration in enumerate(times_per_iteration_list):
            mlflow.log_metric("times_per_iteration_"+str(i), time_per_iteration)

        mlflow.log_metric("final_energy_approx", final_energy_approx)
    
        current_run += 1

Hyperparameter search for N2 in 631g Basis
===

In [ ]:
run_name = "n2_631g"
ezfio_path = '/home/ivan/Descargas/QP_examples/n2/n2_631g.ezfio'
FCI_energy = -109.10842
current_run = 0
for params in hyperparameters_combination:
    print('*****************************************current run:', current_run, 'out of', len(hyperparameters_combination), '********************************************')

    experiment_name = "experiment_cigen_v2"
    experiment = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id
    ezfio_name=ezfio_path.split('/')[-1]

    with mlflow.start_run(experiment_id= experiment_id, run_name=run_name) as run:

        # this two hyperparameters are fixed
        max_iterations = 10  
        qpsh_path = '/home/ivan/Descargas/qp2/bin/qpsh'

        # Set the hyperparameters for the run
        num_epochs, batch_size, learning_rate, k, times_num_dets_gen, prune, tol, temp = params

        # run the model
        ground_energy_list, number_of_det_list, times_per_iteration_list = cigen.main(ezfio_path, qpsh_path, max_iterations, 
            num_epochs, batch_size, learning_rate, k, times_num_dets_gen, prune, tol, temp, FCI_energy)
        
        final_energy_approx = ground_energy_list[-1]

        # save the parameters of the model in MLflow
        mlflow.log_param("max_iterations", max_iterations)
        mlflow.log_param("ezfio_path", ezfio_path)
        mlflow.log_param("num_epochs", num_epochs)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("k", k)
        mlflow.log_param("times_num_dets_gen", times_num_dets_gen)
        mlflow.log_param("prune", prune)
        mlflow.log_param("tol", tol)
        mlflow.log_param("temp", temp)

        

        #save image of the results
        for iteration in range(1, len(ground_energy_list)-1):
            mlflow.log_artifact("graphs/det_distribution/det_distribution_"+ezfio_name+"_iteration_"+str(iteration)+".png")

        mlflow.log_artifact('graphs/energy_plot/'+ezfio_name+'_ground_energy.png')
        mlflow.log_artifact('graphs/time_per_dets/'+ezfio_name+'_time_per_dets.png')

        #save the results of the model in MLflow
        #mlflow.log_metric("ground_energy_list", ground_energy_list)
        for i, ground_energy in enumerate(ground_energy_list):
            mlflow.log_metric("ground_energy_"+str(i), ground_energy)

        #mlflow.log_metric("number_of_det_list", number_of_det_list)
        for i, number_of_det in enumerate(number_of_det_list):
            mlflow.log_metric("number_of_det_"+str(i), number_of_det)
        
        #mlflow.log_metric("times_per_iteration_list", times_per_iteration_list)
        for i, time_per_iteration in enumerate(times_per_iteration_list):
            mlflow.log_metric("times_per_iteration_"+str(i), time_per_iteration)

        mlflow.log_metric("final_energy_approx", final_energy_approx)
    
        current_run += 1

Hyperparameter search for N2 in cc-pvdz Basis
===

In [ ]:
run_name = "n2_ccpvdz"
ezfio_path = '/home/ivan/Descargas/QP_examples/n2/n2_ccpvdz.ezfio'
FCI_energy = -109.27834
current_run = 0
for params in hyperparameters_combination:
    print('*****************************************current run:', current_run, 'out of', len(hyperparameters_combination), '********************************************')

    experiment_name = "experiment_cigen_v2"
    experiment = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id
    ezfio_name=ezfio_path.split('/')[-1]

    with mlflow.start_run(experiment_id= experiment_id, run_name=run_name) as run:

        # this two hyperparameters are fixed
        max_iterations = 10  
        qpsh_path = '/home/ivan/Descargas/qp2/bin/qpsh'

        # Set the hyperparameters for the run
        num_epochs, batch_size, learning_rate, k, times_num_dets_gen, prune, tol, temp = params

        # run the model
        ground_energy_list, number_of_det_list, times_per_iteration_list = cigen.main(ezfio_path, qpsh_path, max_iterations, 
            num_epochs, batch_size, learning_rate, k, times_num_dets_gen, prune, tol, temp, FCI_energy)
        
        final_energy_approx = ground_energy_list[-1]

        # save the parameters of the model in MLflow
        mlflow.log_param("max_iterations", max_iterations)
        mlflow.log_param("ezfio_path", ezfio_path)
        mlflow.log_param("num_epochs", num_epochs)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("k", k)
        mlflow.log_param("times_num_dets_gen", times_num_dets_gen)
        mlflow.log_param("prune", prune)
        mlflow.log_param("tol", tol)
        mlflow.log_param("temp", temp)

        

        #save image of the results
        for iteration in range(1, len(ground_energy_list)-1):
            mlflow.log_artifact("graphs/det_distribution/det_distribution_"+ezfio_name+"_iteration_"+str(iteration)+".png")

        mlflow.log_artifact('graphs/energy_plot/'+ezfio_name+'_ground_energy.png')
        mlflow.log_artifact('graphs/time_per_dets/'+ezfio_name+'_time_per_dets.png')

        #save the results of the model in MLflow
        #mlflow.log_metric("ground_energy_list", ground_energy_list)
        for i, ground_energy in enumerate(ground_energy_list):
            mlflow.log_metric("ground_energy_"+str(i), ground_energy)

        #mlflow.log_metric("number_of_det_list", number_of_det_list)
        for i, number_of_det in enumerate(number_of_det_list):
            mlflow.log_metric("number_of_det_"+str(i), number_of_det)
        
        #mlflow.log_metric("times_per_iteration_list", times_per_iteration_list)
        for i, time_per_iteration in enumerate(times_per_iteration_list):
            mlflow.log_metric("times_per_iteration_"+str(i), time_per_iteration)

        mlflow.log_metric("final_energy_approx", final_energy_approx)
    
        current_run += 1